---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [111]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [103]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [104]:
#%%timeit -n 1
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    '''
    df1 = pd.DataFrame.from_dict(states, orient = 'index')
    df1.columns = ["State"]
    df1 = df1.reset_index().set_index(['State'])
    '''
    
    ## For the heck of experimenting! I'm unnecessarily complicating the code now ...
    df2 = pd.read_table("university_towns.txt", header=None)
    df2.columns = ["Data"]
    stateindexes = df2[df2["Data"].str.contains("\[edit\]")].index
    regionrange= list(zip(stateindexes[::1]+1, stateindexes[1::1]))
    regionrange.append((regionrange[-1][1] + 1, len(df2)))
    regionindex = [[*range(x[0], x[1])] for x in regionrange] 
    Dataframes = []
    for t in range(len(stateindexes)):
        state = df2.iloc[stateindexes[t],:].apply(lambda x : x.split("[")[0])["Data"]
        regions = df2.iloc[regionindex[t],:]["Data"].apply(lambda x : x.split(" (")[0])
        for region in regions:
            Dataframes.append([state, region])
    df = pd.DataFrame(Dataframes)    
    df.columns = ["State", "RegionName"]
    return df
get_list_of_university_towns()

State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas                        Searcy
20      California                        Angwin
21      California                        Arcata
22      California                      Berkeley
23      California                         Chico
24      California                     Claremont
25      California                        Cotati
26      California                         Davis
27      California                        Irvine
28      California                    Isla Vista
29      California  University Park, Los Angeles
..             ...                           ...
487       Virginia                          Wise
488       Virginia                    Chesapeake
489     Washington                    Bellingham
490     Washington                        Cheney
491     Washington                    Ellensburg
492     Washington                       Pullman
493     Washington  University District, Seattle
494  West Virginia                        Athens
495  West Virginia                    Buckhannon
496  West Virginia                      Fairmont
497  West Virginia                     Glenville
498  West Virginia                    Huntington
499  West Virginia                    Montgomery
500  West Virginia                    Morgantown
501  West Virginia                 Shepherdstown
502  West Virginia                  West Liberty
503      Wisconsin                      Appleton
504      Wisconsin                    Eau Claire
505      Wisconsin                     Green Bay
506      Wisconsin                     La Crosse
507      Wisconsin                       Madison
508      Wisconsin                     Menomonie
509      Wisconsin                     Milwaukee
510      Wisconsin                       Oshkosh
511      Wisconsin                   Platteville
512      Wisconsin                   River Falls
513      Wisconsin                 Stevens Point
514      Wisconsin                      Waukesha
515      Wisconsin                    Whitewater
516        Wyoming                       Laramie

[517 rows x 2 columns]

In [105]:
df = pd.read_excel("gdplev.xls")[7:]
dfA = df.iloc[:,0:3]
dfQ = df.iloc[:,4:7]
dfA.reset_index()
dfQ.reset_index()
dfA.columns = ["Year", "GDP($B) current", "GDP($B) chained"]
dfQ.columns = ["Year", "GDP($B) current", "GDP($B) chained"]
dfQ = (dfQ.iloc[212:,:])
quarters = dfQ["Year"]
dfQ.set_index("Year", inplace = True)

In [106]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #print(dfQ)
    consecutivequarters = [*zip(quarters[::1], quarters[1::1], quarters[2::1])]
    for threequarter in consecutivequarters:
        x,y,z = threequarter
        #print("x = {}, y = {}, z = {}".format(x,y,z))
        if (dfQ.loc[x]["GDP($B) chained"] > dfQ.loc[y]["GDP($B) chained"]) and \
        (dfQ.loc[y]["GDP($B) chained"] > dfQ.loc[z]["GDP($B) chained"]):
            return y
    #return dfQ
get_recession_start()

'2008q3'

In [107]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #print(dfQ)
    consecutivequarters = [*zip(quarters[::1], quarters[1::1], quarters[2::1])]
    for threequarter in consecutivequarters:
        x,y,z = threequarter
        #print("x = {}, y = {}, z = {}".format(x,y,z))
        if (dfQ.loc[x]["GDP($B) chained"] < dfQ.loc[y]["GDP($B) chained"]) and \
        (dfQ.loc[y]["GDP($B) chained"] < dfQ.loc[z]["GDP($B) chained"]) and \
        (x > '2008q3'):
            return z
get_recession_end()

'2009q4'

In [108]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    min = 99999999
    for quarter in quarters:
        if min > dfQ.loc[quarter]["GDP($B) chained"] and '2008q2' < quarter and quarter < '2009q4':
            min = dfQ.loc[quarter]["GDP($B) chained"]
            bottom = quarter
    return bottom
get_recession_bottom()

'2009q2'

In [109]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    monthquarterdict = {'01': 'q1', '02': 'q1', '03': 'q1',
                       '04': 'q2', '05': 'q2', '06': 'q2',
                       '07': 'q3', '08': 'q3', '09': 'q3',
                       '10': 'q4', '11': 'q4', '12': 'q4'}
    
    selcols = ["RegionName", "State"] + list(df.columns[[*range(51,251)]]) 
    df1 = df[selcols]
    droppedcols =[]
    for i in range(2,len(selcols)):
        x,y = df1.columns[i].split("-")
        quarter = monthquarterdict[y]
        if (x+quarter) in df1.columns:
            df1[x+quarter] = df1[x+quarter] + df1[x+"-"+y]
            droppedcols.append(x+"-"+y)
        else:
            df1[x+quarter] = df1[x+"-"+y]
            droppedcols.append(x+"-"+y)
    df1.drop(droppedcols, axis = 1, inplace = True)
    df2 = df1[["State","RegionName"]]
    for cols in df1.columns[[*range(2,len(df1.columns))]] :
        if cols == '2016q3':
            df2 = pd.concat([df2, df1[cols].apply(pd.to_numeric)/2], axis = 1)
        else:
            df2 = pd.concat([df2, df1[cols].apply(pd.to_numeric)/3], axis = 1)
    df3 = pd.DataFrame.from_dict(states, orient = 'index')
    df3.columns = ["State"]
    #print(df3)
    #df3 = df3.reset_index().set_index(['State'])
    df2.set_index(["State"], inplace=True)
    df4 = df3.merge(df2, left_index=True, right_index=True)
    df4 = df4.reset_index()
    df4.set_index(["State", "RegionName"],inplace=True)
    df4 = df4.drop(['index'], axis = 1)
    return df4
convert_housing_data_to_quarters()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2000q1         2000q2         2000q3  \
State         RegionName                                                      
Alaska        Anchorage         174633.333333  175266.666667  179566.666667   
              Fairbanks         163200.000000  165033.333333  169300.000000   
              Palmer            168766.666667  170566.666667  174233.333333   
              North Pole        157466.666667  159100.000000  162133.333333   
              Juneau            192466.666667  194300.000000  195166.666667   
              Kenai                       NaN            NaN            NaN   
              Soldotna                    NaN            NaN            NaN   
              Kodiak                      NaN            NaN            NaN   
              Tanaina           160500.000000  163500.000000  166366.666667   
              Ketchikan                   NaN            NaN            NaN   
              Lakes             172733.333333  174133.333333  177266.666667   
              Homer                       NaN            NaN            NaN   
              Seward                      NaN            NaN            NaN   
              Anchor Point                NaN            NaN            NaN   
Alabama       Mobile             78233.333333   79366.666667   80433.333333   
              Birmingham         54033.333333   54400.000000   54966.666667   
              Huntsville                  NaN            NaN            NaN   
              Tuscaloosa        104500.000000  106033.333333  108500.000000   
              Decatur                     NaN            NaN            NaN   
              Hoover            184966.666667  186366.666667  187200.000000   
              Madison                     NaN            NaN            NaN   
              Northport         125866.666667  126366.666667  126633.333333   
              Wetumpka                    NaN            NaN            NaN   
              Fairhope          161400.000000  161933.333333  161633.333333   
              Albertville        73966.666667   72600.000000   72833.333333   
              Alabaster         122133.333333  123066.666667  123166.666667   
              Prichard                    NaN            NaN            NaN   
              Theodore           76900.000000   78300.000000   78066.666667   
              Daphne            123166.666667  120233.333333  122300.000000   
              Homewood          158333.333333  159666.666667  161100.000000   
...                                       ...            ...            ...   
Wisconsin     Holland           151033.333333  150500.000000  153233.333333   
West Virginia Parkersburg        54900.000000   55833.333333   56633.333333   
              Charleston         59533.333333   59366.666667   59366.666667   
              Elkview                     NaN            NaN            NaN   
              South Charleston            NaN            NaN            NaN   
              Vienna             72266.666667   74533.333333   75333.333333   
              Cross Lanes        78133.333333   78066.666667   79233.333333   
              Saint Albans       63833.333333   64700.000000   65200.000000   
              Dunbar                      NaN            NaN            NaN   
              Nitro                       NaN            NaN            NaN   
              Williamstown       82933.333333   85200.000000   86466.666667   
              Clendenin          39966.666667   38200.000000   40500.000000   
              Sissonville        57200.000000   58200.000000   61200.000000   
              Washington         83900.000000   86033.333333   88333.333333   
              Davisville                  NaN            NaN            NaN   
              Mineral Wells      75533.333333   77233.333333   81966.666667   
              Upper Falls        53033.333333   52333.333333   52100.000000   
              Chesapeake                  NaN            NaN            NaN   
          

In [140]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    univstateregion = get_list_of_university_towns()
    univstateregion.set_index(["State", "RegionName"], inplace = True)
    df = convert_housing_data_to_quarters()
    filterdf = df.loc[:,"2000q1":"2009q2"]
    #Hypothesis: University towns have their mean housing prices less effected by recessions. 
    #Run a t-test to compare the ratio of the mean price of houses in university towns the quarter 
    #before the recession starts compared to the recession bottom. 
    #(price_ratio=quarter_before_recession/recession_bottom)
    univstateregionpricesdf = univstateregion.merge(filterdf, left_index = True, right_index = True)
    univstateregionpricesdf["Ratio"] = univstateregionpricesdf.loc[:,"2008q2"] / univstateregionpricesdf.loc[:,"2009q2"]
    nonuniversityregionpricedf = filterdf[~filterdf.index.isin(univstateregion.index)].dropna()
    nonuniversityregionpricedf["Ratio"] = nonuniversityregionpricedf.loc[:,"2008q2"] / nonuniversityregionpricedf.loc[:,"2009q2"]
    stasval, p = ttest_ind(univstateregionpricesdf["Ratio"], nonuniversityregionpricedf["Ratio"], nan_policy ="omit")
    r = univstateregionpricesdf.loc[:,"2008q2"].mean() / nonuniversityregionpricedf.loc[:,"2008q2"].mean()
    #print("statsval = ", stasval)
    #print("p = ", p)
    #print("meanpriceratio - univ/non-univ = ", r)
    return (True, p, "university town")
run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.00070540960968635148, 'university town')